In [1]:
from datasets import load_dataset

In [ ]:
dirs = """
pmg_ga_co/RCPiarsachALL
nnc_ga_mu/CDNamedEntities01

nnc_ga_mu/CDAn_tOileanach_part4_newLTS
nnc_ga_mu/CDNamedEntities02
nnc_ga_mu/CDAlphabet
nnc_ga_mu/CDAn_tOileanach_part2_newLTS
nnc_ga_mu/corpas_beag_newLTS
nnc_ga_mu/CDSaytime

cmg_ga_mu/CMGOther
anb_ga_ul/new_text02
anb_ga_ul/text01-004
anb_ga_ul/text01
anb_ga_ul/text01-001
anb_ga_ul/text02
anb_ga_ul/named-entities
nnc_en_ie
"""

In [34]:
egset = load_dataset("abairxml.py", data_dir=f"/home/jim/tmp/nnc_ga_mu/CDAn_tOileanach_part3_newLTS")

Using custom data configuration default-7a61ec2b352637f4
Reusing dataset utterance_xml_dataset (/home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b)


In [3]:
egset

DatasetDict({
    train: Dataset({
        features: ['file_id', 'words', 'phonemes', 'audio', 'dialect', 'language', 'speaker_id', 'audio_set'],
        num_rows: 1453
    })
})

In [35]:
from pathlib import Path
def check_paths(batch):
    audio_path = Path(batch["audio"])
    return audio_path.is_file()

egset = egset.filter(check_paths)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-65a1ed1f06c66730.arrow


In [36]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["audio"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["words"]
    return batch

egset = egset.map(speech_file_to_array_fn, remove_columns=egset['train'].column_names)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-57b4fd0b447303a2.arrow


In [7]:
egset

DatasetDict({
    train: Dataset({
        features: ['speech', 'sampling_rate', 'target_text'],
        num_rows: 1453
    })
})

In [9]:
import librosa
import numpy as np

In [37]:
def resample(batch):
    batch["duration"] = librosa.get_duration(y=batch["speech"], sr=batch["sampling_rate"])

    if batch["sampling_rate"] != 16_000:
        batch["speech"] = librosa.resample(np.asarray(batch["speech"]), batch["sampling_rate"], 16_000)
        batch["sampling_rate"] = 16_000

    return batch
egset = egset.map(resample, num_proc=4)

Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-020554618c0473da.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-80c4a8247db8a5f0.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-3faf7bd55f7b23b8.arrow
Loading cached processed dataset at /home/jim/.cache/huggingface/datasets/utterance_xml_dataset/default-7a61ec2b352637f4/1.1.0/2ab4d5a39b76b2b213d176be1f94b8bb2c34198ea2fabf4b281e0784782b2d2b/cache-f0ae75f51af33439.arrow


In [38]:
egset = egset.sort('duration')

In [39]:
egset['train'][0]['duration']

1.6585625

In [41]:
egset['train']['duration']

[1.6585625,
 1.7026875,
 1.7631875,
 1.7810625,
 1.8805625,
 1.926625,
 1.950125,
 1.9725625,
 2.0075625,
 2.030125,
 2.0303125,
 2.0375,
 2.03975,
 2.111375,
 2.124875,
 2.1413125,
 2.1545625,
 2.163,
 2.1696875,
 2.1708125,
 2.1816875,
 2.1915625,
 2.198375,
 2.201,
 2.206,
 2.211625,
 2.221625,
 2.2251875,
 2.2401875,
 2.2450625,
 2.2621875,
 2.277,
 2.301625,
 2.3069375,
 2.330625,
 2.330625,
 2.340625,
 2.3535625,
 2.3589375,
 2.359375,
 2.387375,
 2.4046875,
 2.4175,
 2.4189375,
 2.447375,
 2.463125,
 2.4848125,
 2.4880625,
 2.491625,
 2.493875,
 2.5024375,
 2.5053125,
 2.5125,
 2.520375,
 2.52425,
 2.5526875,
 2.5543125,
 2.55825,
 2.586875,
 2.5869375,
 2.5894375,
 2.601625,
 2.617,
 2.625375,
 2.6309375,
 2.6565,
 2.6681875,
 2.6815625,
 2.693375,
 2.7094375,
 2.7235625,
 2.730125,
 2.7303125,
 2.7315,
 2.732,
 2.7326875,
 2.737375,
 2.742125,
 2.7436875,
 2.7461875,
 2.74875,
 2.75175,
 2.7656875,
 2.7686875,
 2.7699375,
 2.775375,
 2.7883125,
 2.7965625,
 2.82825,
 2.8366875

In [40]:
egset['train'][-1]['duration']

14.8091875